In [1]:
import torch 
import torch.nn as nn  
import numpy as np
from tqdm.auto import tqdm
from torch.cuda.amp import autocast
import cv2
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast
from torch.cuda import amp
from torch.optim import lr_scheduler
import time
import gc
import copy
from collections import defaultdict
from typing import Optional, TypeVar
import torch.nn.functional as F
import matplotlib.pyplot as plt
import albumentations as A
from pathlib import Path
from PIL import Image
import segmentation_models_pytorch as smp




import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel

from torch.cuda.amp import autocast
from torch.cuda import amp


#from segmentation.config import Configs as CFG
from segmentation.models.unet import unet
from segmentation.scr.utils import losses, metrics, set_seed, transforms, rle_coding
from segmentation.scr.tilling_dataset import Tilling_Dataset
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

#from segmentation.scr
pd.options.mode.chained_assignment = None

c:\Users\123\Desktop\human vasculature\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\123\AppData\Local\Temp\ipykernel_22388\3577123373.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
class CFG:

    # configs for tilling dataset kidney 1
    path_img_kidney1 = "data\\train\\kidney_1_dense\\images"
    path_lb_kidney1 = "data\\train\\kidney_1_dense\\labels"
    path_df_kidney_1_till = "data\\kidney_1_tilling.csv"
    tile_size = (512, 512)
    overlap_pct = 10
    cache_dir = "data"

    # configs for tilling dataset kidney 1
    path_img_kidney3 = "data\\train\\kidney_3_sparse\\images"
    path_lb_kidney3 = "data\\train\\kidney_3_dense\\labels"
    path_df_kidney_3_till = "data\\kidney_3_tilling.csv"
    tile_size = (512, 512)
    overlap_pct = 10
    cache_dir = "data"

    # configs for transforms
    p_rot = 0.3
    p_aug = 0.3
    # cofigs for train / eval model
    random_seed = 42
    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_batch_size = 4
    n_accumulate = max(1, 32 // train_batch_size)
    valid_batch_size = train_batch_size * 2
    
    max_norm_grad = 5

    epochs = 28
    lr = 3e-4
    
    
    

In [3]:
train_transform  = transforms.get_transform(transform_type='weak')
val_transform = transforms.get_transform(transform_type='val')

In [4]:
train_dataset = Tilling_Dataset(
    name_data='kidney_1_tilling',
    path_to_df=CFG.path_df_kidney_1_till,
    use_random_sub=True,
    empty_tile_pct=10,
    sample_limit=3000
)
val_dataset = Tilling_Dataset(
    name_data='kidney_1_tilling',
    path_to_df=CFG.path_df_kidney_3_till,
    use_random_sub=True,
    empty_tile_pct=10,
    sample_limit=50
)

3000
Dataset contains 1528 empty and 12146 non-empty tiles.
Sample 300 empty and 2700 non-empty tiles.
50
Dataset contains 3420 empty and 4596 non-empty tiles.
Sample 5 empty and 45 non-empty tiles.


In [5]:
set_seed.set_seed(42)

train_loader = DataLoader(train_dataset, batch_size=CFG.train_batch_size, num_workers=2, shuffle=True, pin_memory=True)
val_loader = DataLoader(train_dataset, batch_size=CFG.valid_batch_size, num_workers=2, shuffle=False, pin_memory=True)

In [6]:
model = unet.UNet(n_channels=3, n_classes=1, bilinear=False).to(CFG.device)
num_epoch = 4
loss_fn = losses.BCE_DICE(mode="SUM")
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)
device = CFG.device

In [8]:
def train_one_loop(model, optimizer, loss_func, train_loader, device, grad_clip = True):
    model.train()
    running_loss = 0.0
    epoch_loss = 0.0

    dataset_size = 0
    optimizer.zero_grad()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc='Train ')
    for step, batch in pbar:
        images, masks,_, _ = batch
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        
        images = images / 255.
        masks = masks / 255.
        
        batch_size = images.shape[0]
        dataset_size += batch_size
    
        y_pred = model(images)
            
        loss   = loss_func(y_pred, masks)
        loss   = loss / CFG.n_accumulate
        loss.backward() #loss.backward()  # backward-pass

        running_loss += loss.item() * batch_size
        dataset_size += batch_size
        del images
        del masks
        del y_pred
        
        if (step + 1) % CFG.n_accumulate == 0 or (step + 1 == len(train_loader)):
            if grad_clip:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=CFG.max_norm_grad)
            optimizer.step()  # update weights
            optimizer.zero_grad()



        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix( epoch=f'{step + 1}',
                          train_loss=f'{running_loss / dataset_size:0.4f}',
                          #train_dice = f'{train_dice:0.4f}',
                          #train_jaccard = f'{train_jaccard:0.4f}',
                          lr=f'{current_lr:0.5f}',
                          gpu_mem=f'{mem:0.2f} GB')
    epoch_loss = running_loss / dataset_size

    gc.collect()
    torch.cuda.empty_cache()
    
    return epoch_loss
        
            
        
    
    

In [9]:
train_one_loop(model=model,optimizer=optimizer, train_loader=train_loader,
               loss_func=loss_fn, device=CFG.device)

Train :   2%|▏         | 13/750 [04:09<3:56:12, 19.23s/it, epoch=13, gpu_mem=14.25 GB, lr=0.00030, train_loss=0.0960]


KeyboardInterrupt: 

In [1]:
images, masks,_, _ = next(iter(train_loader))
images = images.to(CFG.device, dtype=torch.float)
masks  = masks.to(CFG.device, dtype=torch.float)

y_pred = model(images)
            

0.2844

In [ ]:
def train_one_epoch_usual(model, dataloader, loss_fn, optimizer, epoch):
    model.train()

    scaler = amp.GradScaler()
    epoch_loss = 0
    running_loss = 0
    train_loss = 0.0
    dataset_size = 0
    train_scores = []

    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    loss_batch = 0
    for step, (images, masks) in pbar:
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        batch_size = images.size(0)
        dataset_size += batch_size

        with amp.autocast():
            y_pred = model(images)
            loss   = loss_fn(y_pred, masks)
            loss   = loss / CFG.n_accumulate




        scaler.scale(loss).backward() #loss.backward()  # backward-pass

        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size
        if (step + 1) % n_accumulate == 0 or (step + 1 == len(dataloader)):
            scaler.step(optimizer) #optimizer.step()  # update weights
            scaler.update()





            #train_loss  += loss_batch  * batch_size * n_accumulate
            #epoch_loss = train_loss / dataset_size
            #loss_batch = 0
            optimizer.zero_grad()


        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']

        #y_pred = nn.Sigmoid()(y_pred)
        #train_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        #train_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        #3train_scores.append([train_dice, train_jaccard])
        #if (step + 1) % n_accumulate == 0 or (step + 1 == len(dataloader)):
        pbar.set_postfix( epoch=f'{step}',
                          #train_loss=f'{loss_batch / len(dataloader):0.4f}',
                          #train_dice = f'{train_dice:0.4f}',
                          #train_jaccard = f'{train_jaccard:0.4f}',
                          lr=f'{current_lr:0.5f}',
                          gpu_mem=f'{mem:0.2f} GB')


    #train_scores  = np.mean(train_scores, axis=0)


    del images
    del masks
    del y_pred
    gc.collect()
    torch.cuda.empty_cache()
    print(epoch_loss)
    return epoch_loss

In [ ]:
model

In [ ]:
epochs = 25

def train():
    model = AttentionUNet()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:

train_loader[2][0]


In [ ]:
train_loader.df

In [ ]:
train_loader[8]

In [ ]:
img_path, lb_path, is_empty, bbx, px_stats, size = train_loader[8]



fig, axarr = plt.subplots(ncols=3, figsize=(12, 6))
axarr[0].imshow(torch.permute(img, (1,2,0)).numpy(), cmap="gray")
axarr[1].imshow(color.label2rgb(mask.numpy(), torch.permute(img, (1,2,0)).numpy(), bg_label=0, bg_color=(1.,1.,1.), alpha=0.25))
axarr[2].imshow(mask, vmin=0, interpolation='antialiased', interpolation_stage='rgba')